In [2]:
% matplotlib inline
# not sure if necessary

UsageError: Line magic function `%` not found.


In [3]:
import finetune
import importlib
import torch

In [4]:
# run this cell when you change stuff in the finetune module, and don't want to restart the notebook's kernel.
importlib.reload(finetune)
finetune.structure_checkpoints()  # If this prints 'Cannot safely create ...', it means that the folder probably already exists

In [6]:
# inception model doesn't work.
# models = ['resnet', 'alexnet', 'densenet', 'squeezenet', 'vgg']
#
# for name in models:
#     print('[TRAINING]:', name)
# finetune.CLASSIFIER_NAME = name
importlib.reload(finetune)

for classifier_name in ['densenet', 'vgg']: # 'resnet',
    for occlusion_name in [None]: # '0', '1', 'GAUSSIAN', 'SOFTMAX'
        for dataset in ['FGVC-Aircraft']: # , 'StanfordCars'
            print('STARTING: model={} | dataset={} | occlusion={}'.format(classifier_name, dataset, occlusion_name))
            finetune.CLASSIFIER_NAME = classifier_name
            finetune.NUM_EPOCHS = 50  # early stopping
            finetune.FEATURE_EXTRACT = False
            finetune.DATASET = dataset  # [StanfordCars, FGVC-Aircraft]
            finetune.OCCLUSION_NAME = occlusion_name
            finetune.OPTIMIZER_NAME = 'adam'
            # finetune.print_hypers() # shows params
            # occlusion = '0'

            # Initialize the model for this run
            model_ft = finetune.initialize_model(_verbose=False)

            model, hist, state = finetune.finetune_model(model_ft, checkpoint_save=10)

            # Save the model after the last epoch
            finetune.save_model(state)
            del model_ft
            del model
            del hist
            del state


STARTING: model=densenet | dataset=FGVC-Aircraft | occlusion=None


Epoch 6/49 train:  68%|██████▊   | 283/417 [00:14<00:06, 19.42it/s, loss=2.0395, accuracy=0.5371] 


KeyboardInterrupt: 

In [ ]:
# # Loading certain model
# model_props = {
#     'name': 'resnet',
#     'dataset': 'StanfordCars',
#     'epochs': 2
# }
# model_load_path = finetune.format_model_path(model_props['name'],
#                                              model_props['dataset'],
#                                              3)
# ,
# state = torch.load(model_load_path)
# new_model = finetune.get_model_architecture(model_props['name'])
# new_model.load_state_dict(state['model_state_dict'])
# optimizer_state_dict = state['optimizer_state_dict']

In [ ]:
# importlib.reload(finetune)

In [ ]:
finetune.save_model(state)

In [ ]:
finetune.format_model_path('resnet',
                           'FGVC-Aircraft',
                           30)

checkpoint = finetune.load_checkpoint(model_save_path)

train_acc_history, train_loss_history, val_acc_history = finetune.get_information_from_checkpoint(checkpoint, plot=True)

In [ ]:
optimizer_state_dict = checkpoint['optimizer_state_dict']
finetune.NUM_EPOCHS = 10
checkpoint_model = finetune.get_model_architecture('resnet')
checkpoint_model.load_state_dict(checkpoint['model_state_dict'])

test_model, hist, state = finetune.finetune_model(checkpoint_model,
                                                  'adam',
                                                  optimizer_state_dict=optimizer_state_dict,
                                                  checkpoint_save=5,
                                                  is_retrain=True)


In [ ]:
hist

In [ ]:
finetune.save_model(state)